In [1]:
import pandas as pd

In [2]:
covid = pd.read_csv('../data/nyt_county.csv')

In [3]:
print(covid.shape)
covid.head()

(1381436, 6)


,date,county,state_name,county_fips_code,confirmed_cases,deaths
0,2021-02-19,Bristol,Rhode Island,44001.0,4115,123.0
1,2021-02-19,Kent,Rhode Island,44003.0,14896,291.0
2,2021-02-19,Providence,Rhode Island,44007.0,80264,1730.0
3,2021-02-19,Chittenden,Vermont,50007.0,4590,83.0
4,2021-02-20,Apache,Arizona,4001.0,10244,365.0


In [4]:
census = pd.read_csv('../data/census_data_2018.csv')

In [5]:
print(census.shape)
census.head()

(3220, 242)


,geo_id,do_date,total_pop,households,male_pop,female_pop,median_age,male_under_5,male_5_to_9,male_10_to_14,...,management_business_sci_arts_employed,sales_office_employed,in_grades_1_to_4,in_grades_5_to_8,in_grades_9_to_12,in_school,in_undergrad_college,speak_only_english_at_home,speak_spanish_at_home,speak_spanish_at_home_low_english
0,35039,2014-01-01,39307,12398,19250,20057,40.6,1224,1240,1278,...,NaN,NaN,2170,1985,1970,9260,1653,NaN,NaN,NaN
1,72133,2014-01-01,22066,7465,10650,11416,37.8,567,662,864,...,2171.0,1242.0,1155,1362,1319,5884,1410,NaN,NaN,NaN
2,72043,2014-01-01,39265,13346,19056,20209,40.7,945,1109,1431,...,3023.0,2529.0,1851,2144,1901,9475,2219,NaN,NaN,NaN
3,72151,2014-01-01,34149,11722,16541,17608,42.5,754,1095,1020,...,1969.0,1700.0,1957,1388,1933,8055,2147,NaN,NaN,NaN
4,72071,2014-01-01,42420,15012,20629,21791,41.6,898,1159,1381,...,3246.0,2725.0,1996,2057,2211,10099,2551,NaN,NaN,NaN


In [24]:
print(census.columns)

Index(['do_date', 'total_pop', 'households', 'male_pop', 'female_pop',
       'median_age', 'male_under_5', 'male_5_to_9', 'male_10_to_14',
       'male_15_to_17',
       ...
       'management_business_sci_arts_employed', 'sales_office_employed',
       'in_grades_1_to_4', 'in_grades_5_to_8', 'in_grades_9_to_12',
       'in_school', 'in_undergrad_college', 'speak_only_english_at_home',
       'speak_spanish_at_home', 'speak_spanish_at_home_low_english'],
      dtype='object', length=241)


In [6]:
by_county = covid.groupby('county_fips_code').max()

In [7]:
census.set_index('geo_id', inplace=True)

In [8]:
df = pd.merge(by_county, census, left_index = True, right_index=True)

In [9]:
print('by_county df:', by_county.shape)
print('census df:', census.shape)
print('final df:', df.shape)

by_county df: (3218, 5)
census df: (3220, 241)
final df: (3211, 246)


In [10]:
df

,date,county,state_name,confirmed_cases,deaths,do_date,total_pop,households,male_pop,female_pop,...,management_business_sci_arts_employed,sales_office_employed,in_grades_1_to_4,in_grades_5_to_8,in_grades_9_to_12,in_school,in_undergrad_college,speak_only_english_at_home,speak_spanish_at_home,speak_spanish_at_home_low_english
1001,2021-06-02,Autauga,Alabama,7161,111.0,2014-01-01,55200,21115,26874,28326,...,9336.0,5331.0,3009,2823,3245,13435,1865,NaN,NaN,NaN
1003,2021-06-02,Baldwin,Alabama,21674,311.0,2014-01-01,208107,78622,101188,106919,...,33923.0,22509.0,9861,11627,10456,45076,6674,NaN,NaN,NaN
1005,2021-06-02,Barbour,Alabama,2340,59.0,2014-01-01,25782,9186,13697,12085,...,2261.0,1729.0,1106,1280,1424,5216,704,NaN,NaN,NaN
1007,2021-06-02,Bibb,Alabama,2666,64.0,2014-01-01,22527,6840,12152,10375,...,1775.0,1531.0,922,1154,1169,4424,757,NaN,NaN,NaN
1009,2021-06-02,Blount,Alabama,6889,139.0,2014-01-01,57645,20600,28434,29211,...,6385.0,4569.0,3064,2998,3180,12220,1718,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72145,2021-06-02,Vega Baja,Puerto Rico,3442,NaN,2014-01-01,53371,18081,25580,27791,...,4179.0,3476.0,2280,3023,2658,12419,2978,NaN,NaN,NaN
72147,2021-06-02,Vieques,Puerto Rico,205,NaN,2014-01-01,8771,2470,4332,4439,...,609.0,423.0,435,441,248,1497,163,NaN,NaN,NaN
72149,2021-06-02,Villalba,Puerto Rico,878,NaN,2014-01-01,22993,7712,11169,11824,...,1578.0,1388.0,1175,1102,1496,6008,1618,NaN,NaN,NaN
72151,2021-06-02,Yabucoa,Puerto Rico,1392,NaN,2014-01-01,34149,11722,16541,17608,...,1969.0,1700.0,1957,1388,1933,8055,2147,NaN,NaN,NaN


In [18]:
df.isna().sum().sort_values(ascending=False)[:20]

speak_spanish_at_home_low_english          3211
pop_15_and_over                            3211
pop_widowed                                3211
pop_separated                              3211
pop_now_married                            3211
pop_never_married                          3211
pop_5_years_over                           3211
speak_only_english_at_home                 3211
speak_spanish_at_home                      3211
pop_divorced                               3211
aggregate_travel_time_to_work               154
population_1_year_and_over                   79
different_house_year_ago_same_city           79
less_than_high_school_graduate               79
graduate_professional_degree                 79
high_school_including_ged                    79
bachelors_degree_2                           79
some_college_and_associates_degree           79
different_house_year_ago_different_city      79
deaths                                       78
dtype: int64

In [13]:
hospital = pd.read_csv('../data/covid19hospitalbycounty.csv')

In [15]:
hospital.sort_values(by='county')

,county,todays_date,hospitalized_covid_confirmed_patients,hospitalized_suspected_covid_patients,hospitalized_covid_patients,all_hospital_beds,icu_covid_confirmed_patients,icu_suspected_covid_patients,icu_available_beds
10319,Alameda,2020-09-29,78.0,47.0,125.0,2390.0,26.0,5.0,126.0
5273,Alameda,2020-07-01,123.0,47.0,170.0,2968.0,43.0,12.0,141.0
14257,Alameda,2020-12-08,242.0,38.0,280.0,2541.0,63.0,5.0,87.0
17494,Alameda,2021-02-04,302.0,42.0,344.0,2520.0,91.0,4.0,67.0
16060,Alameda,2021-01-09,428.0,43.0,471.0,2526.0,115.0,6.0,66.0
...,...,...,...,...,...,...,...,...,...
8212,Yuba,2020-08-22,17.0,1.0,18.0,265.0,4.0,0.0,8.0
17741,Yuba,2021-02-08,36.0,2.0,38.0,261.0,11.0,0.0,5.0
8159,Yuba,2020-08-21,26.0,0.0,26.0,265.0,6.0,0.0,9.0
21017,Yuba,2021-04-08,6.0,1.0,7.0,261.0,0.0,0.0,1.0


In [19]:
mask = pd.read_csv('../data/mask-use-by-county.csv')

In [20]:
mask

,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,1001,0.053,0.074,0.134,0.295,0.444
1,1003,0.083,0.059,0.098,0.323,0.436
2,1005,0.067,0.121,0.120,0.201,0.491
3,1007,0.020,0.034,0.096,0.278,0.572
4,1009,0.053,0.114,0.180,0.194,0.459
...,...,...,...,...,...,...
3137,56037,0.061,0.295,0.230,0.146,0.268
3138,56039,0.095,0.157,0.160,0.247,0.340
3139,56041,0.098,0.278,0.154,0.207,0.264
3140,56043,0.204,0.155,0.069,0.285,0.287
